In [1915]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import re
from collections import defaultdict

In [1916]:
#Use one ./ if running from run_notebooks.py
#Use two ../if running directly from Jupyter Notebooks
#file_beg = '../NHANES-Downloader/data/csv_data/'

file_beg = '../NHANES-Downloader/data/csv_data/'

In [1917]:
#Import all the files
files1 = glob.glob(file_beg+'1999-2000/*/*.csv')
files2 = glob.glob(file_beg+'2001-2002/*/*.csv')
files3 = glob.glob(file_beg+'2003-2004/*/*.csv')
files4 = glob.glob(file_beg+'2005-2006/*/*.csv')
files5 = glob.glob(file_beg+'2007-2008/*/*.csv')
files6 = glob.glob(file_beg+'2009-2010/*/*.csv')
files7 = glob.glob(file_beg+'2011-2012/*/*.csv')
files8 = glob.glob(file_beg+'2013-2014/*/*.csv')
files9 = glob.glob(file_beg+'2015-2016/*/*.csv')
files10 = glob.glob(file_beg+'2017-2018/*/*.csv')

In [1918]:
#Add files into a list
file_list = [files1, files2, files3, files4, files5, 
          files6, files7, files8, files9, files10]

In [1919]:
#Add files into a list of sorted and dictionaries
file_list_dict = []
for x in file_list:
    x.sort()
    file_list_dict.append(dict(enumerate(x)))

In [1920]:
tchol_indx =  [39, 43, 55, 70, 68, 71, 76, 99, 72, 97]

hdl_indx =    [40, 44, 55, 45, 44, 45, 50, 72, 44, 54]

trigly_indx = [40, 44, 56, 72, 71, 76, 79, 102, 75, 99]

for i in range(9,len(hdl_indx)):
    a = pd.read_csv(file_list_dict[i][hdl_indx[i]])

In [1921]:
dfs_tc = defaultdict(int)
dfs_hdl = defaultdict(int)
dfs_tr = defaultdict(int)
for i in range(0,len(tchol_indx)):
    dfs_tc[i] = pd.read_csv(file_list_dict[i][tchol_indx[i]])
for i in range(0,len(hdl_indx)):
    dfs_hdl[i] = pd.read_csv(file_list_dict[i][hdl_indx[i]])
for i in range(0,len(trigly_indx)):
    dfs_tr[i] = pd.read_csv(file_list_dict[i][trigly_indx[i]])

## Join Columns Together

In [1922]:
def innerjoin_df(dfs_list):
    df_join = dfs_list[0]
    for d in dfs_list[1:]:
        df_join = df_join.merge(d, how='inner', on=['SEQN'])
    return df_join

In [1923]:
f = innerjoin_df([dfs_tc[3], dfs_tr[3]])
s = innerjoin_df([f, dfs_hdl[0]])

In [1924]:
dfs = defaultdict(int)
for i in range(0,len(dfs_tc)):
    j = innerjoin_df([dfs_tc[i], dfs_tr[i]])
    if (i>2):
        df_merged = pd.merge(dfs_hdl[i], j, how='inner', left_index=True, right_index=True, suffixes=('', '_drop'))
        df_merged.drop([col for col in df_merged.columns if 'drop' in col], axis=1, inplace=True)
        j = df_merged

    dfs[i] = j

## Important Variables

In [1925]:
#1999-2002
var_names = ['SEQN', 'LBXTC', 'LBDTCSI', 'LBDHDL', 'LBDHDLSI', 'LBXTR', 'LBDTRSI', 'LBDLDL', 'LBDLDLSI']

#2003-2004
var_names1 = ['SEQN', 'LBXTC', 'LBDTCSI', 'LBXHDD', 'LBDHDDSI', 'LBXTR', 'LBDTRSI', 'LBDLDL', 'LBDLDLSI']

#2005-2016
var_names2 = ['SEQN', 'LBXTC', 'LBDTCSI', 'LBDHDD', 'LBDHDDSI', 'LBXTR', 'LBDTRSI', 'LBDLDL', 'LBDLDLSI']

In [1926]:
#To display all columns in Jupyter Notebooks
pd.set_option('display.max_columns', 500)

## Functions: Recategorize values, Count Values, Drop Rows

In [1927]:
#Recategorize function
def recategorize(df, name, replace_dict):
    df[name].replace(
    to_replace=replace_dict,
    inplace=True
)
    
#Count values function
def count_vals(df, name):
    df_count = df.groupby(name)['SEQN'].nunique()
    print(df_count,"\n\n","NaN: ", df[name].isnull().sum())
    
#Drop rows that include certain values
def drop_rows(df, name, val_list):
    df.drop(df[df[name].isin(val_list)].index, inplace=True)

## Make a copy of the dataframes

In [1928]:
df0 = dfs[0][var_names].copy() #1999-2000
df1 = dfs[1][var_names].copy() #2001-2002
df2 = dfs[2][var_names1].copy() #2003-2004
df3 = dfs[3][var_names2].copy() #2005-2006
df4 = dfs[4][var_names2].copy() #2007-2008
df5 = dfs[5][var_names2].copy() #2009-2010
df6 = dfs[6][var_names2].copy() #2011-2012
df7 = dfs[7][var_names2].copy() #2013-2014
df8 = dfs[8][var_names2].copy() #2015-2016
df9 = dfs[9][var_names2].copy() #2018-2019

## Recategorize values

In [1929]:
#Nothing to recategorize

## Rename columns 1999 - 2018

In [1930]:
#New column names
col_names = var_names

In [1931]:
df0.columns = col_names
df1.columns = col_names
df2.columns = col_names
df3.columns = col_names
df4.columns = col_names
df5.columns = col_names
df6.columns = col_names
df7.columns = col_names
df8.columns = col_names
df9.columns = col_names

## Append years 1999 - 2018

In [1932]:
years = ["1999-2000","2001-2002","2003-2004","2005-2006","2007-2008", 
        "2009-2010","2011-2012","2013-2014","2015-2016", "2017-2018"]

In [1933]:
frames = [df0, df1, df2, df3, df4, df5, df6, df7, df8, df9]

In [1934]:
#Add years as a column
for i, df in enumerate(frames):
    df["Year"] = years[i]

In [1935]:
result_1999_2018 = pd.concat(frames, keys = years)

In [1936]:
result_1999_2018_cleaned = result_1999_2018.copy()

In [1937]:
len(result_1999_2018)

35456

## Count values for each column

In [1938]:
len(col_names)

9

In [1939]:
len(result_1999_2018)

35456

In [1940]:
result_1999_2018_cleaned[:5]

SEQN  LBXTC  LBDTCSI  LBDHDL  LBDHDLSI  LBXTR  LBDTRSI  LBDLDL  \
1999-2000 0   2.0  215.0     5.56    54.0      1.39  128.0     1.45   136.0   
          1   3.0  129.0     3.34    30.0      0.78  202.0     2.28    58.0   
          2   5.0  279.0     7.21    42.0      1.08  347.0     3.92   168.0   
          3   7.0  245.0     6.34   105.0      2.73   62.0     0.70   127.0   
          4   8.0  162.0     4.19    67.0      1.74   33.0     0.37    88.0   

             LBDLDLSI       Year  
1999-2000 0      3.52  1999-2000  
          1      1.50  1999-2000  
          2      4.34  1999-2000  
          3      3.28  1999-2000  
          4      2.28  1999-2000

## Remove rows with missing values:

In [1941]:
for i in range (1, len(col_names)):
    drop_rows(result_1999_2018_cleaned, col_names[i], [np.nan])

In [1942]:
len(result_1999_2018_cleaned)

29419

In [1943]:
result_1999_2018_cleaned.isnull().values.any()

False

## See percentage of removed values

In [1944]:
before = len(result_1999_2018)
before

35456

In [1945]:
after = len(result_1999_2018_cleaned)
after

29419

In [1946]:
(before-after)/after

0.20520751895033823

In [1947]:
result_1999_2018_cleaned

SEQN  LBXTC  LBDTCSI  LBDHDL  LBDHDLSI  LBXTR  LBDTRSI  \
1999-2000 0         2.0  215.0     5.56    54.0      1.39  128.0    1.450   
          1         3.0  129.0     3.34    30.0      0.78  202.0    2.280   
          2         5.0  279.0     7.21    42.0      1.08  347.0    3.920   
          3         7.0  245.0     6.34   105.0      2.73   62.0    0.700   
          4         8.0  162.0     4.19    67.0      1.74   33.0    0.370   
...                 ...    ...      ...     ...       ...    ...      ...   
2017-2018 3030  97441.0  116.0     3.00   147.0      3.80   49.0    0.553   
          3032  97443.0  209.0     5.40    90.0      2.33   52.0    0.587   
          3033  97444.0  119.0     3.08    67.0      1.73   78.0    0.881   
          3034  97445.0  172.0     4.45    45.0      1.16   49.0    0.553   
          3035  97446.0  163.0     4.22    46.0      1.19  269.0    3.037   

                LBDLDL  LBDLDLSI       Year  
1999-2000 0      136.0     3.520  1999-2000  
          1       58.0     1.500  1999-2000  
          2      168.0     4.340  1999-2000  
          3      127.0     3.280  1999-2000  
          4       88.0     2.280  1999-2000  
...                ...       ...        ...  
2017-2018 3030    67.0     1.733  2017-2018  
          3032   123.0     3.181  2017-2018  
          3033    43.0     1.112  2017-2018  
          3034   108.0     2.793  2017-2018  
          3035    75.0     1.940  2017-2018  

[29419 rows x 10 columns]

## Remove Duplicated Measurements

In [1948]:
result_1999_2018_cleaned.head()

SEQN  LBXTC  LBDTCSI  LBDHDL  LBDHDLSI  LBXTR  LBDTRSI  LBDLDL  \
1999-2000 0   2.0  215.0     5.56    54.0      1.39  128.0     1.45   136.0   
          1   3.0  129.0     3.34    30.0      0.78  202.0     2.28    58.0   
          2   5.0  279.0     7.21    42.0      1.08  347.0     3.92   168.0   
          3   7.0  245.0     6.34   105.0      2.73   62.0     0.70   127.0   
          4   8.0  162.0     4.19    67.0      1.74   33.0     0.37    88.0   

             LBDLDLSI       Year  
1999-2000 0      3.52  1999-2000  
          1      1.50  1999-2000  
          2      4.34  1999-2000  
          3      3.28  1999-2000  
          4      2.28  1999-2000

In [1949]:
sub_names = list(result_1999_2018.columns)
sub_names

['SEQN',
 'LBXTC',
 'LBDTCSI',
 'LBDHDL',
 'LBDHDLSI',
 'LBXTR',
 'LBDTRSI',
 'LBDLDL',
 'LBDLDLSI',
 'Year']

In [1950]:
mg_dl = [0,1,3,5,7,9]
sub_names = [sub_names[x] for x in mg_dl]

In [1951]:
result_1999_2016_cleaned = result_1999_2018_cleaned[sub_names]

## MongoDB Insertion

In [1952]:
#Import MongoClient
from pymongo import MongoClient

In [1953]:
#Create a MongoClient to run the MongoDB instance
client = MongoClient("localhost", 27017)

In [1954]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [1955]:
#Creating a database
db = client['NHANES']
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'NHANES')

In [1956]:
db.list_collection_names()

['hiq',
 'alq',
 'drxtot',
 'smqfam',
 'smq',
 'tchol',
 'bpq',
 'bmx',
 'huq',
 'bpx',
 'mcq_h',
 'paq',
 'demo_p',
 'mcq_a',
 'mcq_b',
 'rdq',
 'demo',
 'diq',
 'mcq_c']

In [1957]:
#Creating a collection
tchol = db.tchol

In [1958]:
#If collections exist, then drop
if 'tchol' in db.list_collection_names():
    tchol.drop()
    db.list_collection_names()

## Inputting into DB

In [1959]:
result_1999_2016_cleaned.rename(columns= {'SEQN':'_id'}, inplace=True)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [1960]:
result_1999_2016_cleaned[:3]

_id  LBXTC  LBDHDL  LBXTR  LBDLDL       Year
1999-2000 0  2.0  215.0    54.0  128.0   136.0  1999-2000
          1  3.0  129.0    30.0  202.0    58.0  1999-2000
          2  5.0  279.0    42.0  347.0   168.0  1999-2000

In [1961]:
tchol_dict = result_1999_2016_cleaned.to_dict(orient='records')

In [1962]:
tchol_dict[0]

{'_id': 2.0,
 'LBXTC': 215.0,
 'LBDHDL': 54.0,
 'LBXTR': 128.0,
 'LBDLDL': 136.0,
 'Year': '1999-2000'}

In [1963]:
#Insert collection
tchol.insert_many(tchol_dict)

In [1964]:
db.list_collection_names()

['hiq',
 'alq',
 'drxtot',
 'smqfam',
 'smq',
 'bpq',
 'bmx',
 'huq',
 'bpx',
 'mcq_h',
 'tchol',
 'paq',
 'demo_p',
 'mcq_a',
 'mcq_b',
 'rdq',
 'demo',
 'diq',
 'mcq_c']